In [5]:
import opendatasets as od
import pandas
 
od.download("https://www.kaggle.com/competitions/feather-in-focus/data")

['Untitled.ipynb',
 '.jupyter',
 '.config',
 'untitled.py',
 'jupyterhub-2022-4581118.out',
 '.local',
 '.ipynb_checkpoints',
 'jupyterhub-2022-4581898.out',
 'jupyterhub-2022-4581898.error',
 'Untitled1.ipynb',
 '.keras',
 'feather-in-focus.zip',
 '.jupyter-2022',
 '.ipython',
 '.bash_profile',
 '.cache',
 '.bashrc',
 'jupyterhub-2022-4581118.error']

In [3]:
#https://www.kaggle.com/code/chekoduadarsh/starters-guide-convolutional-xgboost/notebook
#https://www.kaggle.com/code/pedrolucasbritodes/bird-image-classification-cnn-89-accuracy/notebook
#https://stats.stackexchange.com/questions/404809/is-it-advisable-to-use-output-from-a-ml-model-as-a-feature-in-another-ml-model

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

#how many epchos still revert
patience1 = 3

base_learning_rate = 0.001 
#number before fine_tuning
number_epcho1 = 20
#number after fine_tuning
number_epcho2 = 20
number_class = 200
#needs to be the same number of classes
output_Neurons = 200
Neurons = 512
drop_out_rate = 0.3
image_shape = (224,224)
image_shape_full = (224,224,3)
current_directory = "feather-in-focus"

In [7]:
#loading files 
class_names2 = np.load('feather-in-focus/class_names.npy', allow_pickle=True).item()

#load all files because why not
test_image_path = pd.read_csv('feather-in-focus/test_images_path.csv')
test_image_path['total_path'] = current_directory + "/test_images" + test_image_path['image_path'] 
test_image_path['total_path'] = test_image_path['total_path'].str.replace("\\", "/" )
test_image = pd.read_csv('feather-in-focus/test_images_sample.csv')

#importing training data 
train_image = pd.read_csv('feather-in-focus/train_images.csv')
train_image['total_path'] = current_directory + "/train_images" + train_image['image_path']
train_image['total_path'] = train_image['total_path'].str.replace("\\", "/" )
train_image['label'] = train_image['label'].apply(str)
train_image.head()

/scratch-local/scur2079.4584660/ipykernel_1454364/3190752840.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  test_image_path['total_path'] = test_image_path['total_path'].str.replace("\\", "/" )
/scratch-local/scur2079.4584660/ipykernel_1454364/3190752840.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train_image['total_path'] = train_image['total_path'].str.replace("\\", "/" )


,image_path,label,total_path
0,/train_images/1.jpg,1,feather-in-focus/train_images/train_images/1.jpg
1,/train_images/2.jpg,1,feather-in-focus/train_images/train_images/2.jpg
2,/train_images/3.jpg,1,feather-in-focus/train_images/train_images/3.jpg
3,/train_images/4.jpg,1,feather-in-focus/train_images/train_images/4.jpg
4,/train_images/5.jpg,1,feather-in-focus/train_images/train_images/5.jpg


In [5]:
# we need to add a k-folds, i have stratified so it works a bit better
train, valid = train_test_split(train_image, 
                    test_size=0.2,
                    stratify = train_image['label'],
                    random_state=420)


In [6]:
# this creates the type of augmentation that is done! 
datagen = ImageDataGenerator(
    #rotation_range=15,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range = 15,
    horizontal_flip = True,
    zoom_range = 0.20)
#look at aug from group chat 

#okay this method seems WAY better. only ever use aug images
train_generator=datagen.flow_from_dataframe(
dataframe=train,
x_col='total_path',
y_col='label',
#classes = train['label'].tolist(),
color_mode = 'rgb',
target_size = image_shape)

valid_generator=datagen.flow_from_dataframe(
dataframe=valid,
x_col='total_path',
y_col='label',
#classes = valid['label'].tolist(),
color_mode = 'rgb',
target_size = image_shape)


Found 3140 validated image filenames belonging to 200 classes.
Found 786 validated image filenames belonging to 200 classes.


In [7]:



# creating the model 
# changing the base model to resnet as requested


base_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=image_shape_full,
    pooling='avg',
    classes=number_class,
    weights='imagenet'
)

# Freeze the ResNet50 model layers
base_model.trainable = False

# Create a Sequential model
# this model has two models in it, we just need to make ours.
model = tf.keras.Sequential()

# Add the base model (ResNET) to the Sequential model
model.add(base_model)

# Global Average Pooling layer
# this layer doesnt work with resnet
#model.add(tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer"))
#model.add(tf.keras.layers.GlobalAveragePooling2D())

#More hidden layers, now with BatchNormalization
model.add(tf.keras.layers.Dense(Neurons, activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(drop_out_rate))

model.add(tf.keras.layers.Dense(Neurons, activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(drop_out_rate))
# Output Dense layer
model.add(tf.keras.layers.Dense(output_Neurons, activation="softmax", name="output-layer"))


# adam optim is apparently the best
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)

# Compile the model 
model.compile(
    loss="categorical_crossentropy",
    #loss = "sparse_categorical_crossentropy",
    optimizer=adam_optimizer,
    metrics=["accuracy"]
)


model.summary()
# this is supposed to help with over-fitting
#Setting the early_stop to avoid overfitting
early_stop = tf.keras.callbacks.EarlyStopping(
    patience=patience1,
    min_delta=0.001,
    restore_best_weights=True,)


history = model.fit(
    train_generator,
    epochs= number_epcho1,
    steps_per_epoch=len(train_generator),
    validation_data=valid_generator,
    validation_steps=int(0.25 * len(valid_generator)),
    callbacks=[early_stop],
)

2023-12-04 17:52:17.782805: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-04 17:52:18.306242: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18209 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 3g.20gb, pci bus id: 0000:31:00.0, compute capability: 8.0


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                        

2023-12-04 17:52:28.253171: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-12-04 17:52:31.072678: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-12-04 17:52:31.117894: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x26e160a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-04 17:52:31.117922: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB MIG 3g.20gb, Compute Capability 8.0
2023-12-04 17:52:31.140635: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-04 17:52:31.372715: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifeti

99/99 [==============================] - 51s 412ms/step - loss: 5.2042 - accuracy: 0.0481 - val_loss: 4.6907 - val_accuracy: 0.0625
Epoch 2/20
99/99 [==============================] - 36s 371ms/step - loss: 3.7978 - accuracy: 0.1583 - val_loss: 3.8277 - val_accuracy: 0.1615
Epoch 3/20
99/99 [==============================] - 36s 366ms/step - loss: 3.2513 - accuracy: 0.2516 - val_loss: 3.2692 - val_accuracy: 0.2552
Epoch 4/20
99/99 [==============================] - 36s 365ms/step - loss: 2.9222 - accuracy: 0.2978 - val_loss: 3.4287 - val_accuracy: 0.2135
Epoch 5/20
99/99 [==============================] - 36s 364ms/step - loss: 2.6484 - accuracy: 0.3417 - val_loss: 3.5354 - val_accuracy: 0.2344
Epoch 6/20
99/99 [==============================] - 36s 366ms/step - loss: 2.4533 - accuracy: 0.3720 - val_loss: 3.3305 - val_accuracy: 0.2292
Epoch 6/40
99/99 [==============================] - 43s 380ms/step - loss: 2.7698 - accuracy: 0.3347 - val_loss: 3.3273 - val_accuracy: 0.2448
Epoch 7/40

In [8]:
#number of base model layers
fine_tune_at = 176

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Now that we will allow some layers to be unfreezed, it's better to decrease the learning rate to avoid dramatic changes in those 
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate/100)

# Compile the model again
model.compile(
    #loss = "sparse_categorical_crossentropy",
    loss="categorical_crossentropy",
    optimizer=adam_optimizer,
    metrics=["accuracy"]
)


early_stop = tf.keras.callbacks.EarlyStopping(
    patience=4,
    min_delta=0.001,
    restore_best_weights=True,)


total_epochs = number_epcho1 + 10

history_fine = model.fit(
                        train_generator,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         steps_per_epoch=len(train_generator),
                         validation_data=valid_generator,
                         validation_steps=int(0.25 * len(valid_generator)),
                        #callbacks=[early_stop]
)


Epoch 6/30
99/99 [==============================] - 43s 377ms/step - loss: 2.3436 - accuracy: 0.4096 - val_loss: 3.2635 - val_accuracy: 0.2812
Epoch 7/30
99/99 [==============================] - 36s 363ms/step - loss: 2.3443 - accuracy: 0.4175 - val_loss: 2.9556 - val_accuracy: 0.2917
Epoch 8/30
99/99 [==============================] - 36s 365ms/step - loss: 2.3611 - accuracy: 0.4137 - val_loss: 3.1611 - val_accuracy: 0.2396
Epoch 9/30
99/99 [==============================] - 36s 363ms/step - loss: 2.3520 - accuracy: 0.4061 - val_loss: 2.9015 - val_accuracy: 0.2917
Epoch 10/30
99/99 [==============================] - 36s 365ms/step - loss: 2.3532 - accuracy: 0.4108 - val_loss: 3.1145 - val_accuracy: 0.2448
Epoch 11/30
99/99 [==============================] - 36s 365ms/step - loss: 2.3833 - accuracy: 0.3920 - val_loss: 2.9748 - val_accuracy: 0.2917
Epoch 12/30
99/99 [==============================] - 36s 365ms/step - loss: 2.3391 - accuracy: 0.4220 - val_loss: 2.7182 - val_accuracy: 0.3

In [9]:



current_directory = "feather-in-focus"
model.save('{}/keras_final.keras'.format(current_directory))


# image folder
folder_path = '{}/test_images/test_images/'.format(current_directory)
# path to model
model_path = '{}/keras_final.keras'.format(current_directory)
# dimensions of images
img_width, img_height = 224, 224



In [4]:
from keras.models import load_model
import keras.utils as image
# load the trained model
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate)
model_path = '{}/keras_final.keras'.format(current_directory)
model = load_model(model_path)
model.compile(loss="categorical_crossentropy",
    optimizer=adam_optimizer,
    metrics=["accuracy"])

2023-12-04 19:03:14.433714: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-04 19:03:14.809359: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18209 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 3g.20gb, pci bus id: 0000:31:00.0, compute capability: 8.0


In [5]:
name = list()
images = []
folder_path = 'feather-in-focus/test_images/test_images'  # Your folder path containing images
for img_name in os.listdir(folder_path):
    img_path = os.path.join(folder_path, img_name)
    # Check if the item is a file (not a directory)
    if os.path.isfile(img_path):
        name.append(img_name)
        img = image.load_img(img_path, target_size=image_shape)
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        images.append(img)
        # Process the image further as needed
    else:
        # Skip directories
        continue
        
# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.predict(images, batch_size=10)
classes_x=np.argmax(classes,axis=1)


2023-12-04 19:03:38.495576: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401


 14/400 [>.............................] - ETA: 4s

2023-12-04 19:03:40.866858: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


400/400 [==============================] - 10s 13ms/step


In [21]:
final_df = pd.DataFrame(
    {'id': name,
     'label_predict': classes_x
    })

final_df['id2'] = "/test_images/"+final_df['id']
final_df['label_predict'] = final_df['label_predict'].astype(int)
final_df = final_df[['label_predict','id2']]

idea = np.sort(train_image['label'].unique())
idea2 = pd.DataFrame(idea)
idea2['index1'] = idea2.index
idea2.columns = ['values', 'index1']
idea2['values'] = idea2['values'].astype(int)

final_df4 = final_df.merge(test_image_path[['image_path','id']], left_on = 'id2', right_on='image_path')

final_df4 = final_df4[['id','label_predict']]

final_df2 = final_df4.merge(idea2, how = 'left', left_on = 'label_predict', right_on = 'index1')
final_df3 = final_df2[['id','values']]
#two issues the class values arent right
final_df3.columns = ['id','label']
final_df3


,id,label
0,2472,117
1,3848,7
2,2025,47
3,910,119
4,3401,85
...,...,...
3995,3540,132
3996,3510,33
3997,3093,48
3998,352,46


In [28]:
final_df3.to_csv('output_folder/output.csv', index=False)

In [30]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi
# Set the path to your Kaggle API credentials file (kaggle.json)
os.environ['KAGGLE_CONFIG_DIR'] = 'kaggle.json'
# Instantiate the Kaggle API
api = KaggleApi()
# Replace 'competition-name' with the competition name (as shown in the competition URL)
competition_name = 'feather-in-focus'
# Replace 'file-path.csv' with the path to your CSV file
file_path = 'feather-in-focus'
# Specify the competition to which you want to upload the file
api.competition_name = competition_name
# Upload the CSV file to the competition
api.dataset_create_new(file_path)

ValueError: Metadata file not found: dataset-metadata.json